<a href="https://colab.research.google.com/github/1antw/Application-of-AI/blob/main/Class02_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import torch.nn as nn

In [ ]:
# grad = False
tensor = torch.rand(3, requires_grad=True)
print(tensor)

y = tensor + 6
print(y)
z = 3*y**2 + 2
print(z)
z = z.mean() #z = z/3
print(z)
z.backward() # dz/d(tensor)
print(tensor.grad) #6(t+6)/3 -> 2*tensor+12

tensor([0.1849, 0.1943, 0.2380], requires_grad=True)
tensor([6.1849, 6.1943, 6.2380], grad_fn=<AddBackward0>)
tensor([116.7602, 117.1085, 118.7390], grad_fn=<AddBackward0>)
tensor(117.5359, grad_fn=<MeanBackward0>)
tensor([12.3699, 12.3886, 12.4761])


In [ ]:
# not scalar(會error，只能放scalar)
'''
tensor = torch.tensor([0.37, 0.58, 0.33], requires_grad=True)
print(tensor)

y = tensor + 6
print(y)
z = 3*y**2 + 2 #通常要做mean, 像在cross entropy上
print(z)
z.backward()
'''

'\ntensor = torch.tensor([0.37, 0.58, 0.33], requires_grad=True)\nprint(tensor)\n\ny = tensor + 6\nprint(y)\nz = 3*y**2 + 2\nprint(z)\nz.backward()\n'

In [ ]:
# grad 
tensor = torch.rand(3, requires_grad = True)
print(tensor)
y = tensor.detach() #train做完截斷，否則在test時會改變(不行)
print(y)

tensor([0.7105, 0.7598, 0.4512], requires_grad=True)
tensor([0.7105, 0.7598, 0.4512])


In [ ]:
# grad 
with torch.no_grad(): #此之下torch不做微分(較常用)
  y = tensor + 2
  print(y)
#圖像鎖前面動後面，語音鎖後面動前面

tensor([2.7105, 2.7598, 2.4512])


In [ ]:
# accumulated gradient
weights = torch.tensor([2., 3., 5., 7.], requires_grad=True)

for epoch in range(5):
  outputs = (3*weights).sum() #wight的gradient會存起來，累加
  outputs.backward()

  print(weights.grad)

tensor([3., 3., 3., 3.])
tensor([6., 6., 6., 6.])
tensor([9., 9., 9., 9.])
tensor([12., 12., 12., 12.])
tensor([15., 15., 15., 15.])


In [ ]:
# empty gradient
weights = torch.tensor([2., 3., 5., 7.], requires_grad=True)

for epoch in range(5):
  outputs = (3*weights).sum()
  outputs.backward()

  print(weights.grad)

  weights.grad.zero_() #清除weight的值，要記得清零

tensor([3., 3., 3., 3.])
tensor([3., 3., 3., 3.])
tensor([3., 3., 3., 3.])
tensor([3., 3., 3., 3.])
tensor([3., 3., 3., 3.])


In [ ]:
# toy example
x = torch.tensor([[1,-1], [2,3], [5,2]], dtype=torch.float32) # 3x2
y = torch.tensor([[1],[0],[1]], dtype=torch.float32)

w1 = torch.rand(2,3, requires_grad=True) # 2x3
w2 = torch.rand(3,3, requires_grad=True) # 3x3
w3 = torch.rand(3,2, requires_grad=True) # 3x2
w4 = torch.rand(2,1, requires_grad=True) # 2x1
relu = nn.ReLU()
sigmoid = nn.Sigmoid()
bce = nn.BCELoss()

def forward(inputs):
    inputs = torch.matmul(inputs, w1) # matrix multiply
    inputs = relu(inputs)
    inputs = torch.matmul(inputs, w2)
    inputs = relu(inputs)
    inputs = torch.matmul(inputs, w3)
    inputs = relu(inputs)
    inputs = torch.matmul(inputs, w4)
    outputs = sigmoid(inputs)
    return outputs

# loss
def loss(y_true, y_pred):
    return bce(y_pred, y_true)


learning_rate = 0.01
epochs = 10

for epoch in range(epochs):
    # forward pass
    y_hat = forward(inputs=x)

    # loss
    bce_loss = loss(y_true=y, y_pred=y_hat)

    # backward loss
    bce_loss.backward()

    # update weights
    w1 -= learning_rate * w1.grad
    w2 -= learning_rate * w2.grad
    w3 -= learning_rate * w3.grad
    w4 -= learning_rate * w4.grad

    # zero gradients
    w1.grad.zero_()
    w2.grad.zero_()
    w3.grad.zero_()
    w4.grad.zero_()

    if epoch % 5 == 0:
        print(f"epoch {epoch + 1}: \nw1 = {w1}\n w2 = {w2}\n w3 = {w3}\n w4 = {w4}, loss = {bce_loss:.8f}")